# Matrix Product State

In [1]:
using CUDA
using ITensors, ITensorMPS, Plots

In [2]:
a = 1
ga = 0.5
ma = 0.25
g = ga/a
m_org = ma/a
m_lat = m_org - a*g^2/8
# x = 1/(a^2*g^2)
# mu = 2*m_lat/(a*g^2)

0.21875

In [31]:
N = 10
sites = siteinds("S=1/2", N)

os = OpSum()
for n in 1:N-1
    os += (1/(4*a)), "X", n,"X", n+1
    os += (1/(4*a)), "Y", n,"Y", n+1
end

for n in 1:N
    os += (-1)^n*m_lat/2, "Z", n
end

for n in 1:N-1
    
end

for n in 1:N-1
    for k in 1:n
        os += g^2*a/4*(-1)^k, "Z", k
        os += g^2*a/4, "Id", 1
    end
    for j in 1:n
        for i in 1:j-1
            os += g^2*a/4, "Z", i, "Z", j
            os += g^2*a/4*(-1)^(i+j), "Id", 1
            os += g^2*a/4*(-1)^j, "Z", i
            os += g^2*a/4*(-1)^i, "Z", j
        end
    end
end

H = cu(MPO(os, sites))

MPO
[1] ((dim=5|id=829|"Link,l=1"), (dim=2|id=361|"S=1/2,Site,n=1")', (dim=2|id=361|"S=1/2,Site,n=1"))
[2] ((dim=5|id=829|"Link,l=1"), (dim=5|id=478|"Link,l=2"), (dim=2|id=736|"S=1/2,Site,n=2")', (dim=2|id=736|"S=1/2,Site,n=2"))
[3] ((dim=5|id=478|"Link,l=2"), (dim=5|id=55|"Link,l=3"), (dim=2|id=837|"S=1/2,Site,n=3")', (dim=2|id=837|"S=1/2,Site,n=3"))
[4] ((dim=5|id=55|"Link,l=3"), (dim=5|id=371|"Link,l=4"), (dim=2|id=736|"S=1/2,Site,n=4")', (dim=2|id=736|"S=1/2,Site,n=4"))
[5] ((dim=5|id=371|"Link,l=4"), (dim=5|id=522|"Link,l=5"), (dim=2|id=426|"S=1/2,Site,n=5")', (dim=2|id=426|"S=1/2,Site,n=5"))
[6] ((dim=5|id=522|"Link,l=5"), (dim=5|id=264|"Link,l=6"), (dim=2|id=738|"S=1/2,Site,n=6")', (dim=2|id=738|"S=1/2,Site,n=6"))
[7] ((dim=5|id=264|"Link,l=6"), (dim=5|id=319|"Link,l=7"), (dim=2|id=923|"S=1/2,Site,n=7")', (dim=2|id=923|"S=1/2,Site,n=7"))
[8] ((dim=5|id=319|"Link,l=7"), (dim=5|id=570|"Link,l=8"), (dim=2|id=612|"S=1/2,Site,n=8")', (dim=2|id=612|"S=1/2,Site,n=8"))
[9] ((dim=5|id=57

In [25]:
nsweeps = 30
maxdim = [10,20,100,100,200]
cutoff = [1E-10]

psi0 = cu(random_mps(sites, linkdims = 10))
# 

MPS
[1] ((dim=2|id=487|"S=1/2,Site,n=1"), (dim=10|id=843|"Link,l=1"))
[2] ((dim=10|id=843|"Link,l=1"), (dim=2|id=440|"S=1/2,Site,n=2"), (dim=10|id=161|"Link,l=2"))
[3] ((dim=10|id=161|"Link,l=2"), (dim=2|id=251|"S=1/2,Site,n=3"), (dim=10|id=159|"Link,l=3"))
[4] ((dim=10|id=159|"Link,l=3"), (dim=2|id=313|"S=1/2,Site,n=4"), (dim=10|id=907|"Link,l=4"))
[5] ((dim=10|id=907|"Link,l=4"), (dim=2|id=8|"S=1/2,Site,n=5"), (dim=10|id=830|"Link,l=5"))
[6] ((dim=10|id=830|"Link,l=5"), (dim=2|id=239|"S=1/2,Site,n=6"), (dim=10|id=199|"Link,l=6"))
[7] ((dim=10|id=199|"Link,l=6"), (dim=2|id=432|"S=1/2,Site,n=7"), (dim=8|id=435|"Link,l=7"))
[8] ((dim=8|id=435|"Link,l=7"), (dim=2|id=804|"S=1/2,Site,n=8"), (dim=4|id=672|"Link,l=8"))
[9] ((dim=4|id=672|"Link,l=8"), (dim=2|id=698|"S=1/2,Site,n=9"), (dim=2|id=175|"Link,l=9"))
[10] ((dim=2|id=175|"Link,l=9"), (dim=2|id=428|"S=1/2,Site,n=10"))


In [26]:
energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)

After sweep 1 energy=-3.0899515  maxlinkdim=10 maxerr=8.66E-04 time=0.058
After sweep 2 energy=-3.098713  maxlinkdim=20 maxerr=1.14E-07 time=0.063
After sweep 3 energy=-3.0987155  maxlinkdim=29 maxerr=2.28E-07 time=0.057
After sweep 4 energy=-3.0987186  maxlinkdim=26 maxerr=6.34E-08 time=0.081
After sweep 5 energy=-3.0987191  maxlinkdim=24 maxerr=2.06E-07 time=0.068
After sweep 6 energy=-3.0987215  maxlinkdim=24 maxerr=4.72E-08 time=0.060
After sweep 7 energy=-3.098718  maxlinkdim=24 maxerr=1.26E-07 time=0.054
After sweep 8 energy=-3.0987203  maxlinkdim=26 maxerr=2.66E-08 time=0.060
After sweep 9 energy=-3.0987172  maxlinkdim=21 maxerr=1.76E-08 time=0.052
After sweep 10 energy=-3.0987177  maxlinkdim=24 maxerr=1.82E-07 time=0.056
After sweep 11 energy=-3.0987167  maxlinkdim=29 maxerr=2.64E-08 time=0.080
After sweep 12 energy=-3.098717  maxlinkdim=27 maxerr=1.55E-07 time=0.052
After sweep 13 energy=-3.098721  maxlinkdim=22 maxerr=1.88E-07 time=0.052
After sweep 14 energy=-3.0987175  maxl

(-3.098718f0, MPS
[1] ((dim=2|id=1|"Link,l=1"), (dim=2|id=487|"S=1/2,Site,n=1"))
[2] ((dim=4|id=725|"Link,l=2"), (dim=2|id=440|"S=1/2,Site,n=2"), (dim=2|id=1|"Link,l=1"))
[3] ((dim=2|id=251|"S=1/2,Site,n=3"), (dim=8|id=218|"Link,l=3"), (dim=4|id=725|"Link,l=2"))
[4] ((dim=2|id=313|"S=1/2,Site,n=4"), (dim=7|id=599|"Link,l=4"), (dim=8|id=218|"Link,l=3"))
[5] ((dim=2|id=8|"S=1/2,Site,n=5"), (dim=27|id=908|"Link,l=5"), (dim=7|id=599|"Link,l=4"))
[6] ((dim=2|id=239|"S=1/2,Site,n=6"), (dim=15|id=977|"Link,l=6"), (dim=27|id=908|"Link,l=5"))
[7] ((dim=2|id=432|"S=1/2,Site,n=7"), (dim=8|id=264|"Link,l=7"), (dim=15|id=977|"Link,l=6"))
[8] ((dim=2|id=804|"S=1/2,Site,n=8"), (dim=4|id=191|"Link,l=8"), (dim=8|id=264|"Link,l=7"))
[9] ((dim=2|id=698|"S=1/2,Site,n=9"), (dim=2|id=82|"Link,l=9"), (dim=4|id=191|"Link,l=8"))
[10] ((dim=2|id=428|"S=1/2,Site,n=10"), (dim=2|id=82|"Link,l=9"))
)

In [12]:
cuPsi = cu(psi)

MPS
[1] ((dim=2|id=574|"Link,l=1"), (dim=2|id=263|"S=1/2,Site,n=1"))
[2] ((dim=4|id=724|"Link,l=2"), (dim=2|id=281|"S=1/2,Site,n=2"), (dim=2|id=574|"Link,l=1"))
[3] ((dim=2|id=523|"S=1/2,Site,n=3"), (dim=8|id=795|"Link,l=3"), (dim=4|id=724|"Link,l=2"))
[4] ((dim=2|id=301|"S=1/2,Site,n=4"), (dim=14|id=117|"Link,l=4"), (dim=8|id=795|"Link,l=3"))
[5] ((dim=2|id=837|"S=1/2,Site,n=5"), (dim=7|id=626|"Link,l=5"), (dim=14|id=117|"Link,l=4"))
[6] ((dim=2|id=192|"S=1/2,Site,n=6"), (dim=34|id=128|"Link,l=6"), (dim=7|id=626|"Link,l=5"))
[7] ((dim=2|id=320|"S=1/2,Site,n=7"), (dim=67|id=928|"Link,l=7"), (dim=34|id=128|"Link,l=6"))
[8] ((dim=2|id=723|"S=1/2,Site,n=8"), (dim=43|id=117|"Link,l=8"), (dim=67|id=928|"Link,l=7"))
[9] ((dim=2|id=944|"S=1/2,Site,n=9"), (dim=83|id=45|"Link,l=9"), (dim=43|id=117|"Link,l=8"))
[10] ((dim=2|id=691|"S=1/2,Site,n=10"), (dim=45|id=142|"Link,l=10"), (dim=83|id=45|"Link,l=9"))
[11] ((dim=2|id=703|"S=1/2,Site,n=11"), (dim=81|id=575|"Link,l=11"), (dim=45|id=142|"Link,l

In [13]:
psi_contracted = contract(psi)

Contraction resulted in ITensor with 14 indices, which is greater
            than or equal to the ITensor order warning threshold 14.
            You can modify the threshold with macros like `@set_warn_order N`,
            `@reset_warn_order`, and `@disable_warn_order` or functions like
            `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and
            `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors C:\Users\aaaab\.julia\packages\ITensors\FpnkY\src\tensor_operations\tensor_algebra.jl:20
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors C:\Users\aaaab\.julia\packages\ITensors\FpnkY\src\tensor_operations\tensor_algebra.jl:74
  [3] #213
    @ C:\Users\aaaab\.julia\packages\ITensors\FpnkY\src\tensor_operations\tensor_algebra.jl:158 [inlined]
  [4] BottomRF
    @ .\reduce.jl:86 [inlined]
  [5] _foldl_impl
    @ .\reduce.jl:62 [inlined]
  [6] foldl_impl
    @ .\reduce.jl:48 [inlined]
  [7] mapfoldl_impl
  

ITensor ord=20 (dim=2|id=263|"S=1/2,Site,n=1") (dim=2|id=281|"S=1/2,Site,n=2") (dim=2|id=523|"S=1/2,Site,n=3") (dim=2|id=301|"S=1/2,Site,n=4") (dim=2|id=837|"S=1/2,Site,n=5") (dim=2|id=192|"S=1/2,Site,n=6") (dim=2|id=320|"S=1/2,Site,n=7") (dim=2|id=723|"S=1/2,Site,n=8") (dim=2|id=944|"S=1/2,Site,n=9") (dim=2|id=691|"S=1/2,Site,n=10") (dim=2|id=703|"S=1/2,Site,n=11") (dim=2|id=795|"S=1/2,Site,n=12") (dim=2|id=601|"S=1/2,Site,n=13") (dim=2|id=766|"S=1/2,Site,n=14") (dim=2|id=599|"S=1/2,Site,n=15") (dim=2|id=202|"S=1/2,Site,n=16") (dim=2|id=208|"S=1/2,Site,n=17") (dim=2|id=333|"S=1/2,Site,n=18") (dim=2|id=495|"S=1/2,Site,n=19") (dim=2|id=536|"S=1/2,Site,n=20")
NDTensors.Dense{ComplexF32, CuArray{ComplexF32, 1, CUDA.DeviceMemory}}

# Quantum Number Conserving

In [17]:
siteqnc = siteinds("S=1/2", 5, conserve_qns = true)

5-element Vector{Index{Vector{Pair{QN, Int64}}}}:
 (dim=2|id=543|"S=1/2,Site,n=1") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=626|"S=1/2,Site,n=2") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=103|"S=1/2,Site,n=3") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=44|"S=1/2,Site,n=4") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=575|"S=1/2,Site,n=5") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1